In [65]:
from igraph import *
import numpy as np

In [32]:
file = open('Aluno_Area_Pesquisa.txt', 'r').read()
lines = file.split('\n')
alunos = []
for line in lines:
    line = line.split(' ')
    codigo_aluno = int(line[0])
    area_pesquisa = int(line[0])
    a = Aluno(codigo_aluno, area_pesquisa)
    alunos.append(a)

In [71]:
file = open('Matriz_Dissimilaridade.txt', 'r').read()
lines = file.split('\n')
matriz_triangulo = []
for l in lines:
    l = l.split(' ')
    l = list(filter(None, l))
    matriz_triangulo.append(l)
matriz = completa_matriz(matriz_triangulo)
print(matriz)

[[ 0. 80. 10.]
 [80.  0. 80.]
 [10. 80.  0.]]


In [70]:
def completa_matriz(matriz_triangulo):
    matriz_triangulo.sort(key=len, reverse=True)
    lado = len(matriz_triangulo)
    matriz = np.zeros((lado, lado))
    for i in range(lado):
        for j in range(len(matriz_triangulo[i])):
            matriz[i][i+j] = matriz_triangulo[i][j]
            matriz[i+j][i] = matriz_triangulo[i][j]
    return matriz
    

In [26]:
a = Aluno(1, 2)
a

In [30]:
class Aluno(object):
    def __init__(self, codigo, area):
        self.codigo_aluno = codigo
        self.area_pesquisa = area
    
    def __repr__(self):
        to_string = "Código Aluno: " + str(self.codigo_aluno)
        to_string += ", Área de Pesquisa: " + str(self.area_pesquisa)
        return to_string
        